# TensorFlow Recommenders Implementation
By TFUG Surabaya Team (Joan Santoso, Billy K., Patrick S.)


Import semua library yang diperlukan. Pada tutorial ini library yang diperlukan adalah tensorflow recommenders, numpy, dan tensorflow dataset. <br>
tensorflow digunakan sebagai framework deep learning <br>
numpy merupakan library komputasi matematika di python <br>
tensorflow dataset(tfds) digunakan untuk load dataset yang akan dipakai yaitu MovieLens 100K dataset.
Dataset dapat di download di https://grouplens.org/datasets/movielens/100k/

### Import TFRS

Install library-library yang dibutuhkan, seperti tensorflow-recommenders, dan tensorflow-datasets (kalau sudah ada, tidak perlu untuk menjalankan cell ini).

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.3 MB/s eta 0:00:00a 0:00:01


Import library-library yang dibutuhkan, seperti numpy, tensorflow, tensorflow_datasets, dan tensorflow_recommenders.

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

Ambil data-data yang ingin dijadikan bahan training. Pada kasus ini akan menggunakan movielens.

Dalam kasus ini, kami akan menggunakan tensorflow datasets sebagai bantuan untuk melakukan load file yang diinginkan, di mana tensorflow datasets telah menyediakan datasetnya.

Setelah melakukan load, untuk ratings hanya akan diambil movie_title dan user_id. Sedangkan untuk movies, hanya akan diambil movie_title.

In [ ]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteN82LAO/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


Setelah itu, dari data yang telah di load akan dibuat vocabulary baru untuk user_id dan movie_title. Proses pembuatan vocabulary akan menggunakan bantuan StringLookup yang dimiliki oleh keras.

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

Buat model two-tower recommender systemnya dengan inherit tfrs.Model. Sehingga hanya perlu untuk mengimplementasi init dan compute_loss. Untuk train_step dan test_step sudah sudah dibuatkan oleh tfrs.Model.

Model two-tower membutuhkan 2 model, yaitu user model dan movie model.

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Setelah membuat two-tower model, kita perlu untuk menginisialisasi user model, movie model serta task yang digunakan untuk menghitung loss dari two-tower model.

User_model disini berfungsi sebagai embedding layer untuk user (user_id), sedangkan movie_model berfungsi sebagai embedding layer untuk movie (movie_title). Masing-masing model akan menggunakan vocabulary yang telah dibuat sebelumnya sebagai patokan vocabularynya.

Untuk task menghitung loss, kami menggunakan Retrieval task yang telah disediakan oleh tensorflow-recommender. Di mana metrics yang digunakan merupakan FactorizedTopK yang juga telah disediakan oleh tensorflow-recommender.

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)


### Fit and evaluate it.



Setelah membuat struktur two-tower model serta menginisialisasi model-model dan task yang dibutuhkan, barulah masuk ke fase training. Data yang digunakan dalam fase training hanyalah ratings.

Untuk mendapatkan rekomendasi pada suatu kandidat, dalam kasus ini kami menggunakan brute-force search di mana sudah disediakan oleh tensorflow. Nantinya akan di cari rekomendasi, di mana hasil embedding dari user akan dibandingkan dengan embedding movie-movie yang ada, lalu diambil yang memiliki kemiripan dalam bentuk array.

In [ ]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")